In [1]:
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go

In [35]:


output_dir_gathered_together = "./Facescape/Train/"

subj_idx = 100

tmp = np.load(os.path.join(output_dir_gathered_together, "train_neutral.npy"), allow_pickle=True)

#print(tmp.keys()) # Se vuoi vedere le chaivi del dizionario.

face = tmp['point_list'][subj_idx]
landmark_gt = tmp['landmark_list'][subj_idx]
filename= tmp['subject_list'][subj_idx]
heatmaps_list = tmp['heatmaps_list'][subj_idx]
emotion_list = tmp['emotion_list'][subj_idx]
scale_list = tmp['scale_list'][subj_idx]
normal_list = tmp['normal_list'][subj_idx]

print("Face: ", face.shape)
print("Landmark: ", landmark_gt.shape)
print("Heatmaps: ", heatmaps_list.shape)
print("Normal: ", normal_list.shape)
print("Filename: ", filename)
print("Emotion: ", emotion_list)
print("Scale: ", scale_list)

fig = px.scatter_3d(x=face[:,0], y=face[:,1], z=face[:,2], width=1280, height=720)
fig.update_traces(marker=dict(size=2, color='black'))
fig.add_trace(go.Scatter3d(x=landmark_gt[:, 0], y=landmark_gt[:, 1], z=landmark_gt[:, 2], mode='markers', marker=dict(size=5, color='green')))
fig1 = go.Figure(data=fig.data)
fig1.show()

Face:  (8192, 3)
Landmark:  (68, 3)
Heatmaps:  (8192, 68)
Normal:  (8192, 3)
Filename:  258
Emotion:  1_neutral
Scale:  0.00653478


In [2]:
%ls

DataLoader.py        __pycache__/         main.py
Facescape/           check_dataset.ipynb  req1.txt
Preprocessing/       configuration.py     requirements.txt
README.md            lie_learn/           se3/


In [3]:
#from configuration import *
from DataLoader import FaceLandmarkDataset
import os

train_dataset = FaceLandmarkDataset(
     preprocessing = "none",                         #choices=['icp', 'spatial_transformer', 'none']
     break_ds_with = "none",                         #choices=['rotation', 'translation', 'rotation_translation', 'none']
     split="Train", 
     ds_path=os.path.join("./Facescape"),
     category = "Neutral",
     references_pointclouds_icp_path = "./Preprocessing/reference_pointclouds_for_icp"
)

/Users/mattia/miniconda3/envs/cv-dgl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset...
Path file:  ./Facescape/Train/train_neutral.npy
Preprocessing: none
Dataset: Facescape
Category: Neutral


Len dataset:  (669, 8192, 3)
Face:  (669, 8192, 3)
Landmark:  (669, 68, 3)
Heatmaps:  (669, 8192, 68)
Filename:  (669,)
Scale:  (669,)
Emotion:  {'1_neutral'}


In [4]:
x, y  = train_dataset[0]
print(x)

Graph(num_nodes=8192, num_edges=327680,
      ndata_schemes={'x': Scheme(shape=(1, 3), dtype=torch.float32), 'v': Scheme(shape=(1, 3), dtype=torch.float32)}
      edata_schemes={'d': Scheme(shape=(3,), dtype=torch.float32)})


In [10]:
from se3.model import SE3Transformer


m = SE3Transformer(
    num_layers = 2,
    num_channels = 1
)
#print(m)

m(x)



RuntimeError: mat1 and mat2 shapes cannot be multiplied (327680x1 and 2x32)

In [9]:
from dgl.geometry import farthest_point_sampler
import torch

landmark_number = 30

#heatmap = heatmaps_list[:, landmark_number] 
print(train_dataset[0]['heatmap'].shape)
heatmap = train_dataset[0]['heatmap'][:, landmark_number]
print(heatmap.shape)
face = train_dataset[0]['face']

#print(heatmap[heatmap.argsort()[::-1][:42]])


#print(face.shape)
#pt = torch.tensor(face.reshape(-1, 8192, 3))
#point_idx = farthest_point_sampler(pt, 4096)

#face = face[point_idx[0]]

# Create a 3D scatter plot of the face point cloud
fig = px.scatter_3d(x=face[:, 0], y=face[:, 1], z=face[:, 2], width=1280, height=720)

# Set marker properties for the face point cloud
fig.update_traces(marker=dict(size=2, color='black'))

# corresponding landmark
#fig.add_trace(go.Scatter3d(x=np.array(landmark_gt[landmark_number, 0]), y=np.array(landmark_gt[landmark_number, 1]), z=np.array(landmark_gt[landmark_number, 2]), mode='markers', marker=dict(size=10, color='red')))

# Create a scatter plot for the heatmap colors
fig.add_trace(go.Scatter3d(
    x=face[:, 0],
    y=face[:, 1],
    z=face[:, 2],
    mode='markers',
    marker=dict(size=5, color=heatmap),
    hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>color: %{marker.color}'
))

fig.update_layout(width=1024, height=1024)
# remove bg color
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# remove grid
#fig.update_xaxes(showgrid=False)
#fig.update_yaxes(showgrid=False)
#fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)

# Show the plot
fig.show()

torch.Size([8192, 68])
torch.Size([8192])
